import important library

In [ ]:
from tools_function import *
from varaince_computation import *

MOSI can work with single cluster or multiple cluster at a paraleel way.

We will start with single cluster first.

For the wrapper function, we need to specify the model, temperature, marker list, tissue, species, and additional info. Reccommend to use gpt-4o and temperature 0. If you have special request, you can specify the additional info. Such as you want to look at metastatic tissue, you can specify the additional info as "Consider the possibility of metastatic tissue".

For a single cluster the estimated time is around 30 seconds no matter how many markers you have.


In [2]:
#first define the marker list
marker_list=['IFITM10','RP11-727F15.11','ZFP64','TTN','UBOX5','COG6','MXD1','FAM219A','ST14','SMYD5']
#then run the  wrapper function, specify the model, temperature, marker list, tissue, species, and additional info
results=run_cell_type_analysis(model="gpt-4o", temperature=0, marker_list=marker_list, tissue="bone marrow", species="human", additional_info=None)
results

({'main_cell_type': 'mixed cell population',
  'sub_cell_types': ['Mesenchymal Stem Cells (MSCs)',
   'Epithelial-like Cells',
   'Immune-related Cells'],
  'possible_mixed_cell_types': ['hematopoietic and non-hematopoietic cells'],
  'num_markers': 10,
  'iterations': 2},
 [('Final Annotation Agent',
   'To annotate the cell type based on the provided ranked marker list from a single-cell human dataset derived from bone marrow tissue, I will follow the outlined steps:\n\n### Step 1: List the Key Functional Markers\n1. **IFITM10**: Part of the interferon-induced transmembrane protein family, which is involved in immune response.\n2. **ZFP64**: A zinc finger protein that may play a role in transcriptional regulation.\n3. **TTN**: Encodes titin, a protein important for muscle elasticity and structure, but also expressed in other cell types.\n4. **UBOX5**: Involved in protein ubiquitination, a process that tags proteins for degradation.\n5. **COG6**: Part of the conserved oligomeric Golgi

In [1]:
results

NameError: name 'results' is not defined

For the results, it will return a tuple with two elements. The first element is a dictionary with the main results and related information. The second element is a dictionary that store the conversation history.

In [4]:
results[0]

[('Final Annotation Agent',
  'To annotate the cell type based on the provided ranked marker list from a single-cell human dataset derived from bone marrow tissue, I will follow the outlined steps:\n\n### Step 1: List the Key Functional Markers\n1. **IFITM10**: Part of the interferon-induced transmembrane protein family, which is involved in immune response.\n2. **ZFP64**: A zinc finger protein that may play a role in transcriptional regulation.\n3. **TTN**: Encodes titin, a protein important for muscle elasticity and structure, but also expressed in other cell types.\n4. **UBOX5**: Involved in protein ubiquitination, a process that tags proteins for degradation.\n5. **COG6**: Part of the conserved oligomeric Golgi complex, involved in Golgi function and protein glycosylation.\n6. **MXD1**: A transcriptional repressor that can modulate cell proliferation and differentiation.\n7. **SMYD5**: A histone methyltransferase involved in chromatin modification and gene expression regulation.\n\

In [5]:
results[1]

[('Final Annotation Agent',
  'To annotate the cell type based on the provided ranked marker list from a single-cell human dataset derived from bone marrow tissue, I will follow the outlined steps:\n\n### Step 1: List the Key Functional Markers\n1. **IFITM10**: Part of the interferon-induced transmembrane protein family, which is involved in immune response.\n2. **ZFP64**: A zinc finger protein that may play a role in transcriptional regulation.\n3. **TTN**: Encodes titin, a protein important for muscle elasticity and structure, but also expressed in other cell types.\n4. **UBOX5**: Involved in protein ubiquitination, a process that tags proteins for degradation.\n5. **COG6**: Part of the conserved oligomeric Golgi complex, involved in Golgi function and protein glycosylation.\n6. **MXD1**: A transcriptional repressor that can modulate cell proliferation and differentiation.\n7. **SMYD5**: A histone methyltransferase involved in chromatin modification and gene expression regulation.\n\

We also provide ways to access the accuracy of the reuslts. First run the analsyis for at least 10 times, then run wrapper fucntion to get the similarity score between results and get the consensus annottaion. This will improve the accuracy of the results, and give you a sense of the stability of the results.

The analsyis implement parallel computing, so you can specify the max_workers to control the number of cores you want to use. If you have enough cores, the estimated time will be similar to running a single analysis. If you are not sure about the number of cores, use code "os.cpu_count()" to check.


In [3]:
#specify the number of times you want to run the analysis, and the max_workers is the number of cores you want to use.
results3=run_analysis_n_times(n=3, tissue="bone marrow", species="human", additional_info=None, temperature=0, marker_list=marker_list, model="gpt-4o", max_workers=10)

NameError: name 'run_analysis_n_times' is not defined

In [2]:
results2

NameError: name 'results2' is not defined

In [9]:
varaince=process_cell_type_analysis_single(results2)


The results will returen a dictionary with the following keys:
"unified_results": the consensus annotation
"similarity_score": the similarity score between the each round of results...


In [10]:
varaince

{'unified_results': "result1:('hematopoietic lineage restricted progenitor cell', 'hematopoietic stem cell'),result2:('hematopoietic multipotent progenitor cell', 'common myeloid progenitor'),result3:('hematopoietic stem cell', 'hematopoietic stem cell'),result4:('progenitor cell', 'hematopoietic stem cell'),result5:('mesenchymal stem cell', 'progenitor cell'),result6:('progenitor cell', 'progenitor cell'),result7:('hematopoietic cell', 'hematopoietic stem cell'),result8:('myeloid lineage restricted progenitor cell', 'intermediate monocyte'),result9:('hematopoietic multipotent progenitor cell', 'common myeloid progenitor'),result10:('mesenchymal stem cell', 'mesenchymal stem cell')",
 'consensus_types': ('hematopoietic progenitor cell',
  'Common Myeloid Progenitor (CMP)'),
 'general_celltype_llm': 'progenitor cell',
 'sub_celltype_llm': 'hematopoietic stem cell',
 'Possible_mixed_celltypes_llm': ['progenitor cell', 'mesenchymal stem cell'],
 'similarity_score': 0.2,
 'original_results

Next we will show how to do the analysis for multiple clusters at the same time. The computation is parallelized, so the estimated time is similar to running a single cluster analysis.

the run_cell_type_analysis_batchrun function is used to do the analysis for multiple clusters at the same time. You need to provide the path of the dataframe, and the output json name. The function will return a json file with the results and corresponding csv file for better visualization. 

The dataframe should have two columns, one is the cell type column, and the other is the marker list column. If not specified, the first column will be used as the cell type column and the second column will be used as the marker list column. This dataframe can be directly from common DE method like Findallmarekr in seurat or in scanpy. 

In [11]:
run_cell_type_analysis_batchrun(df_path="C:/Users/ellio/OneDrive - UW-Madison/cellgpt_final_folder/Test_results/Elliot/GTEX/lung_modified_markers_broad_cell_type_50genes.csv", output_json_name="lung_cell_type_analysis_results.json", model="gpt-4o", temperature=0, tissue="lung", species="human", additional_info=None, celltype_column=None, gene_column_name=None, max_workers=10)


Analyzing Endothelial cell (lymphatic)...
Markers: ['CCL21', 'MMRN1', 'PKHD1L1', 'PROX1', 'TBX1', 'LRCOL1', 'GPR182', 'TFF3', 'FLT4', 'HOXD3', 'HOXD8', 'PARD6G', 'ADGRG3', 'GRAPL', 'SCN3B', 'FOXC2', 'SEMA3D', 'KALRN', 'TLL1', 'SEMA3A', 'ZNF521', 'FLRT2', 'LYVE1', 'C6orf141', 'SEMA6A', 'AC108938.5', 'PDE7B', 'STK32B', 'KRTAP10-7', 'RHOJ', 'TSHZ2', 'RELN', 'PRDM8', 'KLHL4', 'KLHL14', 'PGM5', 'OAF', 'CXorf36', 'SNCG', 'LDB2', 'PEAR1', 'KDR', 'TAL1', 'ECSCR', 'FAM43A', 'ST6GALNAC3', 'SMOC1', 'NR2F1', 'C2CD4C', 'CNTNAP3B']

Analyzing Endothelial cell (vascular)...
Markers: ['PTPRB', 'CALCRL', 'SOX17', 'ACKR1', 'BTNL9', 'EGFL7', 'TMEM100', 'CLDN5', 'RAMP3', 'LDB2', 'CLEC14A', 'HEY1', 'SLCO2A1', 'FCN3', 'CYYR1', 'IGFBP3', 'SEMA3G', 'ADGRL4', 'LRRC32', 'CD93', 'VWF', 'RAMP2', 'GATA2', 'S1PR1', 'SHANK3', 'SSTR1', 'GRASP', 'ANO2', 'SLCO4A1', 'KLF2', 'MT1M', 'ARHGEF15', 'CDH5', 'GPIHBP1', 'SELP', 'TMEM204', 'EPAS1', 'VCAM1', 'HYAL2', 'NES', 'MYCT1', 'CLEC1A', 'FOXF1', 'SLC9A3R2', 'NOTCH4', 'TMTC

{'Epithelial cell (alveolar type I)': {'analysis_result': {'main_cell_type': 'Alveolar Type I Cells',
   'sub_cell_types': ['Lung Epithelial Cells', 'Endothelial Cells'],
   'possible_mixed_cell_types': [],
   'num_markers': 50,
   'iterations': 1},
  'conversation_history': [('Final Annotation Agent',
    'To annotate the cell type based on the provided ranked marker list from a single-cell RNA sequencing dataset of human lung tissue, we will follow the outlined steps:\n\n### Step 1: List the Key Functional Markers\n- **RTKN2**: Involved in Rho GTPase signaling, which is important for cytoskeletal organization.\n- **CAV1**: Encodes caveolin-1, a scaffolding protein within caveolae membranes, involved in signal transduction and lipid metabolism.\n- **ST6GALNAC5**: Involved in glycosylation processes.\n- **AGER**: Encodes the receptor for advanced glycation end-products, involved in inflammatory responses.\n- **LAMA3**: Part of the laminin family, important for cell adhesion and tissue 

We also provided function to access the variance of the reuslts. The same marker dataframe need to provided. And a output name need to be specified.

In [13]:
results = run_batch_analysis_n_times(
     n=3,
     df_path="C:/Users/ellio/OneDrive - UW-Madison/cellgpt_final_folder/Test_results/Elliot/GTEX/lung_modified_markers_broad_cell_type_50genes.csv",
     output_json_prefix="GTEX_cell_type_analysis_results_batch_lung",
     model="gpt-4o",
     temperature=0,
     tissue="lung",
     species="human",
     max_workers=10,
     batch_max_workers=10,
 )

Starting batch run 1/3
Starting batch run 2/3
Starting batch run 3/3

Analyzing Endothelial cell (lymphatic)...
Markers: ['CCL21', 'MMRN1', 'PKHD1L1', 'PROX1', 'TBX1', 'LRCOL1', 'GPR182', 'TFF3', 'FLT4', 'HOXD3', 'HOXD8', 'PARD6G', 'ADGRG3', 'GRAPL', 'SCN3B', 'FOXC2', 'SEMA3D', 'KALRN', 'TLL1', 'SEMA3A', 'ZNF521', 'FLRT2', 'LYVE1', 'C6orf141', 'SEMA6A', 'AC108938.5', 'PDE7B', 'STK32B', 'KRTAP10-7', 'RHOJ', 'TSHZ2', 'RELN', 'PRDM8', 'KLHL4', 'KLHL14', 'PGM5', 'OAF', 'CXorf36', 'SNCG', 'LDB2', 'PEAR1', 'KDR', 'TAL1', 'ECSCR', 'FAM43A', 'ST6GALNAC3', 'SMOC1', 'NR2F1', 'C2CD4C', 'CNTNAP3B']

Analyzing Endothelial cell (lymphatic)...
Markers: ['CCL21', 'MMRN1', 'PKHD1L1', 'PROX1', 'TBX1', 'LRCOL1', 'GPR182', 'TFF3', 'FLT4', 'HOXD3', 'HOXD8', 'PARD6G', 'ADGRG3', 'GRAPL', 'SCN3B', 'FOXC2', 'SEMA3D', 'KALRN', 'TLL1', 'SEMA3A', 'ZNF521', 'FLRT2', 'LYVE1', 'C6orf141', 'SEMA6A', 'AC108938.5', 'PDE7B', 'STK32B', 'KRTAP10-7', 'RHOJ', 'TSHZ2', 'RELN', 'PRDM8', 'KLHL4', 'KLHL14', 'PGM5', 'OAF', 'CXor

Then use the following function to process the reuslts and save the full results with variance to csv file.
for the file pattern, you need to specify the format of the result file name. An example is provided below:

results_getx_*_full.csv

In [2]:
process_and_save_batch_results(marker_file_path="C:/Users/ellio/OneDrive - UW-Madison/cellgpt_final_folder/Test_results/Elliot/GTEX/lung_modified_markers_broad_cell_type_50genes.csv", file_pattern="C:/Users/ellio/OneDrive - UW-Madison/cellgpt_final_folder/Test_results/Elliot/GTEX/final_testing/GTEX_cell_type_analysis_results_batch_lung_*_full.csv", output_csv_name="C:/Users/ellio/OneDrive - UW-Madison/cellgpt_final_folder/Test_results/Elliot/GTEX/final_testing/GTEX_cell_type_analysis_results_batch_lung_full.csv")

To determine the consensus cell type, we need to analyze the given results:

1. **Result 1**: ('B cell', 'plasma cell')
2. **Result 2**: ('B cell', 'plasma cell')
3. **Result 3**: ('B cell', 'plasma cell')

**Step-by-step reasoning:**

- All three results indicate the same general cell type: 'B cell'.
- All three results also indicate the same sub cell type: 'plasma cell'.
- There is no variation in the results, which means there is no evidence of mixed cell types.

Given the consistency across all results, the consensus general cell type is 'B cell', and the consensus sub cell type is 'plasma cell'. There is no indication of mixed cell types.

**Final Answer:**

```json
{
  "final_results": [
    "B cell",
    "plasma cell"
  ],
  "possible_mixed_celltypes": []
}
```
B cell plasma cell []
To determine the consensus cell type, let's analyze the given results:

1. **General Cell Type**: All three results indicate the same general cell type: "fallopian tube secretory epithelial cell". Th

,Cell Type,General Cell Type LLM,Sub Cell Type LLM,Possible Mixed Cell Types LLM,General Type Standard,Sub Type Standard,Similarity Score,Unified Results,Original Non-Unified Results,Original General Type 1,Original Sub Type 1,Original General Type 2,Original Sub Type 2,Original General Type 3,Original Sub Type 3
0,Immune (B cell),B cell,plasma cell,,B cell,plasma cell,1.000000,"result1:('B cell', 'plasma cell'),result2:('B ...","result1:('B cells', 'Plasma Cells'),result2:('...",B cells,Plasma Cells,B cells,Plasma Cells,B cells,Plasma Cells
1,Epithelial cell (club),fallopian tube secretory epithelial cell,club cell,,fallopian tube secretory epithelial cell,club cell,1.000000,result1:('fallopian tube secretory epithelial ...,"result1:('secretory epithelial cells', 'club c...",secretory epithelial cells,club cells,secretory epithelial cells,Club Cells (Clara Cells),secretory epithelial cells,club cells
2,Endothelial cell (lymphatic),endothelial cell of lymphatic vessel,endothelial cell of lymphatic vessel,,endothelial cell of lymphatic vessel,endothelial cell of lymphatic vessel,1.000000,result1:('endothelial cell of lymphatic vessel...,"result1:('lymphatic endothelial cells', 'lymph...",lymphatic endothelial cells,lymphatic endothelial cells,lymphatic endothelial cells,Lymphatic Endothelial Cells (LECs),Lymphatic Endothelial Cells,Lymphatic Endothelial Cells
3,Fibroblast,fibroblast,kidney interstitial fibroblast,,fibroblast,kidney interstitial fibroblast,0.900000,"result1:('fibroblast', 'kidney interstitial fi...","result1:('Fibroblasts', 'Interstitial Fibrobla...",Fibroblasts,Interstitial Fibroblasts,fibroblasts,lung fibroblasts,Fibroblasts,Interstitial Fibroblasts
4,Endothelial cell (vascular),endothelial cell,microvascular endothelial cell,,endothelial cell,microvascular endothelial cell,0.900000,"result1:('endothelial cell', 'microvascular en...","result1:('Endothelial Cells', 'Microvascular E...",Endothelial Cells,Microvascular Endothelial Cells,Endothelial Cells,Microvascular Endothelial Cells,endothelial cells,vascular endothelial cells
5,Immune (DC/macrophage),myeloid cell,macrophage,,myeloid cell,macrophage,0.900000,"result1:('myeloid cell', 'macrophage'),result2...","result1:('myeloid cells', 'macrophages'),resul...",myeloid cells,macrophages,myeloid cells,macrophages,myeloid cells,conventional dendritic cells
6,Epithelial cell (basal),epithelial cell,basal epithelial cell of tracheobronchial tree,,epithelial cell,basal epithelial cell of tracheobronchial tree,0.900000,"result1:('epithelial cell', 'basal epithelial ...","result1:('Epithelial Cells', 'Basal Epithelial...",Epithelial Cells,Basal Epithelial Cells,Epithelial,Basal Cells,Epithelial Cells,Basal Epithelial Cells
7,Epithelial cell (ciliated),ciliated epithelial cell,multiciliated variant,,ciliated epithelial cell,multi-ciliated epithelial cell,0.800000,"result1:('ciliated epithelial cell', 'multi-ci...","result1:('ciliated epithelial cells', 'Multici...",ciliated epithelial cells,Multiciliated Cells,Ciliated Epithelial Cells,Ciliated Respiratory Epithelial Cells,ciliated epithelial cells,multiciliated cells (respiratory epithelium)
8,Epithelial cell (alveolar type II),pulmonary alveolar epithelial cell,pulmonary alveolar type 2 cell,,pulmonary alveolar epithelial cell,pulmonary alveolar type 2 cell,0.736667,"result1:('pulmonary alveolar type 2 cell', 'ep...","result1:('Alveolar Type II Cells', 'Epithelial...",Alveolar Type II Cells,Epithelial Cells,Alveolar Epithelial Cells,Alveolar Type II Cells (AT2),Alveolar Epithelial Cells,Alveolar Type II (AT2) Cells
9,Epithelial cell (alveolar type I),epithelial cell,pulmonary alveolar type 1 cell,,epithelial cell,pulmonary alveolar type 1 cell,0.766667,"result1:('pulmonary alveolar epithelial cell',...","result1:('alveolar epithelial cells', 'Alveola...",alveolar epithelial cells,Alveolar Type I Cells,Epithelial Cells,Alveolar Type I Cells,Epithelial Cells,Alveolar Type I Cells
